In [1]:
import pandas as pd
import numpy as np

## Zbiór treningowy

In [2]:
train_data_final_df = pd.read_csv(
    "C:/Users/Ada/Desktop/SRD_projekt_2025/Data/Final/train_data_final_df.csv"
).drop(columns=["Unnamed: 0"])

## Czyszczenie danych 
#### Wstępna analiza zbioru - identyfikacja typów danych

In [3]:
train_data_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439301 entries, 0 to 439300
Data columns (total 34 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Store                                 439301 non-null  int64  
 1   Dept                                  439301 non-null  int64  
 2   Date                                  439301 non-null  object 
 3   Weekly_Sales                          439301 non-null  float64
 4   IsHoliday                             439301 non-null  bool   
 5   Temperature                           439301 non-null  float64
 6   Fuel_Price                            439301 non-null  float64
 7   MarkDown1                             156569 non-null  float64
 8   MarkDown2                             113996 non-null  float64
 9   MarkDown3                             142120 non-null  float64
 10  MarkDown4                             140309 non-null  float64
 11  

| Typ zmiennej           | Liczba zmiennych | Przykłady zmiennych |
|------------------------|-----------------|-------------------|
| Liczbowe całkowite (`int64`) | **13**              | Store, Dept, Size, sams_club_competition_gt, costco_competition_gt, back_to_school_gt ... |
| Liczbowe zmiennoprzecinkowe (`float64`) | **14**              | Weekly_Sales, Temperature, Fuel_Price, Markdown1, Markdown2...|
| Tekstowe (`object`)            | **6**              | Date, Type, valentines_day_gt, black_friday_gt, cyber_monday_gt, HolidayName|
| Logiczna (`bool`)              | **1**               | IsHoliday |



*Uwagi wynikające z analizy typów zmiennych:*

- Zmienna Date jest typu object - a powinna być `datetime`  
- Niektóre kolumny Google 
Trends (valentines_day_gt, black_friday_gt, cyber_monday_gt) są typu `object`, a powinny być numeryczne

#### Exploracja danych: sprawdzenie unikalnych wartości w zmiennych kategorycznych

In [4]:
object_variables = ['Type', 'valentines_day_gt', 'black_friday_gt', 'cyber_monday_gt', 'HolidayName']
for o in object_variables:
    print(f'Zmienna: {train_data_final_df[o].value_counts().sort_index()}\n')

Zmienna: Type
A    224530
B    170369
C     44402
Name: count, dtype: int64

Zmienna: valentines_day_gt
1     100374
11      2918
12      2959
2       5984
20      2919
22      2956
25      2952
3       5946
4       2930
41      2926
46      5915
5       2973
54      2955
7       5894
94      3001
<1    285699
Name: count, dtype: int64

Zmienna: black_friday_gt
1     127116
11      2930
14      2954
2      20756
24      2928
28      2965
3       2959
4      11866
5       2964
72      2916
96      2961
<1    255986
Name: count, dtype: int64

Zmienna: cyber_monday_gt
0      90922
1       8878
2       2928
20      2916
31      2961
4       2965
62      2938
94      3021
<1    321772
Name: count, dtype: int64

Zmienna: HolidayName
Christmas Day                   5986
Columbus Day                    8870
Good Friday                    17722
Independence Day                8781
Indigenous Peoples' Day         8870
Labor Day                       8837
Lincoln's Birthday              8882
Mart

#### Konwersja typów zmiennych

- `Date` → typ `datetime`  
- Zmienne GT (`valentines_day_gt`, `black_friday_gt`, `cyber_monday_gt`) → numeryczne  
  - wartości `'<1'` zamieniono na `0.5`


In [5]:
train_data_final_df['Date'] = pd.to_datetime(train_data_final_df['Date'])
train_data_final_df['Date']

gt_columns = ['valentines_day_gt', 'black_friday_gt', 'cyber_monday_gt']
for g in gt_columns:
    train_data_final_df[g] = train_data_final_df[g].replace('<1', '0.5')
    
    train_data_final_df[g] = pd.to_numeric(train_data_final_df[g])


## Analiza braków danych

In [6]:
train_data_final_df.isna().sum().loc[lambda x: x > 0]

MarkDown1      282732
MarkDown2      325305
MarkDown3      297181
MarkDown4      298992
MarkDown5      281932
HolidayName    321191
dtype: int64

### Analiza braków danych w zmiennych `MarkDown1`–`MarkDown5`

Zmienne `MarkDown1`–`MarkDown5` odzwierciedlają wartość promocji wprowadzonych w danym sklepie w konkretnym tygodniu.  
Występujące w nich wartości `NaN` nie oznaczają błędu pomiarowego, lecz informują o **braku promocji w danym tygodniu**, co jest naturalną sytuacją w danych.

Aby ujednolicić reprezentację tej informacji, wartości `NaN` zostały zastąpione wartością `0`, oznaczającą brak promocji.


In [7]:
markdowns = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
for m in markdowns:
    train_data_final_df[m] = train_data_final_df[m].fillna(0)

#### Analiza braków danych w zmiennej `HolidayName`

Zmienna `HolidayName` określa nazwę święta przypadającego na dany tydzień. Braki oznaczają brak świąt, co jest naturalną sytuacją w danych, a nie błędem pomiarowym.

Aby ujednolicić reprezentację tej informacji, brakujące wartości `NaN` uzupełniono etykietą `NoHoliday`

In [8]:
train_data_final_df['HolidayName'] = train_data_final_df['HolidayName'].fillna('NoHoliday')

Po zastosowanych przekształceniach, zbiór danych pozbawiony został braków danych we wszystkich zmiennych

#### Zmienna `IsHoliday` - zmienna typu `bool`

Konwersja wartości logicznych na wartości binarne: `False` → 0 oraz `True` → 1. 

In [9]:
train_data_final_df['IsHoliday'].value_counts()

IsHoliday
False    409640
True      29661
Name: count, dtype: int64

In [10]:
train_data_final_df['IsHoliday'] = train_data_final_df['IsHoliday'].astype(int)

## Wstępne przetwarzanie danych 

#### Kodowanie zmiennych kategorycznych

Zmienne kategoryczne `Type` i `HolidayName` zostały zakodowane przy użyciu **one-hot encoding**, czyli przekształcono każdą kategorię w osobną kolumnę binarną (0/1).  


In [ ]:
one_hot_encoding = ['Type', 'HolidayName']

for o in one_hot_encoding:
    dummies = pd.get_dummies(train_data_final_df[o], prefix=o)
    dummies.columns = [c.replace(" ", "_").replace("'", "_") for c in dummies.columns]
    train_data_final_df = pd.concat([train_data_final_df, dummies], axis=1)


In [20]:
for i in range (0, len(train_data_final_df.columns)):
    print(train_data_final_df.columns[i])

len(train_data_final_df.columns)

Store
Dept
Date
Weekly_Sales
IsHoliday
Temperature
Fuel_Price
MarkDown1
MarkDown2
MarkDown3
MarkDown4
MarkDown5
CPI
Unemployment
Type
Size
valentines_day_gt
sams_club_competition_gt
costco_competition_gt
black_friday_gt
cyber_monday_gt
back_to_school_gt
dollar_tree_competition_gt
walmart_promotions_gt
kroger_competition_gt
supermarket_near_me_gt
supermarket_gt
target_shop_competition_gt
walmart_gt
Retail_Trade_Not_Seasonally_Adjusted
Retail_Trade_Seasonally_Adjusted
m2_money_supply_indicator
GDP
HolidayName
Type_A
Type_B
Type_C
HolidayName_Christmas_Day
HolidayName_Columbus_Day
HolidayName_Good_Friday
HolidayName_Independence_Day
HolidayName_Indigenous_Peoples__Day
HolidayName_Labor_Day
HolidayName_Lincoln_s_Birthday
HolidayName_Martin_Luther_King,_Jr._Day
HolidayName_Memorial_Day
HolidayName_New_Year_s_Day
HolidayName_NoHoliday
HolidayName_Thanksgiving_Day
HolidayName_Truman_Day
HolidayName_Veterans_Day
HolidayName_Washington_s_Birthday


52

#### Tworzenie zmiennych czasowych 

opisać to ładnie po co, czemu

Zakres dat dla poszczególnych sklepów:

In [27]:
store_date_ranges = train_data_final_df.groupby('Store')['Date'].agg(['min','max'])

# Sprawdzenie, czy min i max są takie same dla wszystkich sklepów
same_range = (store_date_ranges['min'].nunique() == 1) and (store_date_ranges['max'].nunique() == 1)
print("Czy wszystkie sklepy mają ten sam zakres dat?", same_range)


Czy wszystkie sklepy mają ten sam zakres dat? True


In [ ]:
train_data_final_df['year'] = train_data_final_df['Date'].dt.year
train_data_final_df['month'] = train_data_final_df['Date'].dt.month
train_data_final_df['week'] = train_data_final_df['Date'].dt.week
train_data_final_df['day_of_week'] = train_data_final_df['Date'].dt.day_of_week

### Zbiór danych po przetworzeniu zmiennych kategorycznych

Po zastosowaniu **one-hot encoding** dla zmiennych kategorycznych `Type` i `HolidayName` powstał zbiór danych zawierający zarówno zmienne pierwotne, jak i dodatkowe kolumny binarne odpowiadające poszczególnym kategoriom cechy. 


## SPRAWDZIĆ CZY ZAKRES DAT DLA KAZDEGO SKLEPU EJST TAKI SAM
PRZECZYTAC TO CO KONRAD PISAL BO MOZE MNIE W COS WKOPAŁ XD

## UJEDNOLICIĆ SPOSÓB PISANA TYCH KOM I WGL

In [15]:
train_data_final_df.describe()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,walmart_promotions_gt,kroger_competition_gt,supermarket_near_me_gt,supermarket_gt,target_shop_competition_gt,walmart_gt,Retail_Trade_Not_Seasonally_Adjusted,Retail_Trade_Seasonally_Adjusted,m2_money_supply_indicator,GDP
count,439301.000000,439301.000000,439301,439301.000000,439301.000000,439301.000000,439301.000000,439301.000000,439301.000000,439301.00000,...,439301.000000,439301.000000,439301.000000,439301.000000,439301.000000,439301.000000,439301.000000,439301.000000,439301.000000,439301.000000
mean,22.200887,44.256127,2011-06-21 02:24:02.340445184,15960.538310,0.067519,60.024806,3.364524,2572.653007,845.237446,449.36538,...,10.658899,51.209922,1.020717,73.662505,41.082504,31.082654,372351.821127,372466.741130,9294.355000,15617.146714
min,1.000000,1.000000,2010-02-07 00:00:00,-4988.940000,0.000000,-2.060000,2.472000,0.000000,-265.760000,-29.10000,...,0.000000,29.000000,0.000000,61.000000,25.000000,18.000000,304401.000000,339580.000000,8502.300000,14764.610000
25%,11.000000,18.000000,2010-10-10 00:00:00,2083.000000,0.000000,46.990000,2.931000,0.000000,0.000000,0.00000,...,0.000000,37.000000,0.000000,68.000000,36.000000,24.000000,353272.000000,356215.000000,8724.200000,15309.474000
50%,22.000000,37.000000,2011-06-19 00:00:00,7605.530000,0.000000,61.900000,3.467000,0.000000,0.000000,0.00000,...,0.000000,55.000000,0.000000,73.000000,39.000000,31.000000,373236.000000,375442.000000,9131.000000,15557.539000
75%,33.000000,74.000000,2012-02-26 00:00:00,20176.670000,0.000000,73.880000,3.743000,2826.570000,1.300000,4.33000,...,0.000000,60.000000,0.000000,78.000000,44.000000,34.000000,388382.000000,388156.000000,9843.100000,16068.805000
max,45.000000,99.000000,2012-10-28 00:00:00,693099.360000,1.000000,100.140000,4.468000,88646.760000,104519.540000,141630.61000,...,100.000000,79.000000,49.000000,97.000000,100.000000,87.000000,443079.000000,397681.000000,10289.000000,16420.419000
std,12.785623,30.491662,NaN,22669.442538,0.250918,18.222492,0.460792,5981.141966,4983.769270,5416.91608,...,23.556380,11.984684,6.238981,7.124539,10.769857,9.122295,26964.794490,17639.468003,567.275612,486.018259
